In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

In [2]:
model_name = "google/flan-t5-large"
token = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [15]:
chat_flow = [
    "Hi! I am Your Hiring Assistant",
    "What is your full name? ",
    "What is your mobile number? ",
    "What is your email address? ",
    "How many years of experience do you have? ",
    "What position are you applying for? ",
    "Tech Stack: what are your technical skills (languages, frameworks, databases, tools)? "
]

In [20]:
# Collect user data
user_data = {}
print(chat_flow[0])
user_data["name"] = input(chat_flow[1])
user_data["mobile"] = input(chat_flow[2])
user_data["email"] = input(chat_flow[3])
user_data["exp"] = input(chat_flow[4])
user_data["jobrole"] = input(chat_flow[5])
raw_stack = [t.strip().lower() for t in input(chat_flow[6]).split(",") if t.strip()]
# Filter valid tech (exclude tools like VSCode)
valid_tech = {
    "languages": ["python", "javascript", "java", "c++", "ruby"],
    "frameworks": ["django", "flask", "react", "angular", "spring"],
    "databases": ["sql", "mysql", "postgresql", "mongodb"]
}
tech_stack = [t for t in raw_stack if t in sum(valid_tech.values(), [])]
user_data["teck_stack"] = ", ".join(tech_stack) if tech_stack else "python"
tech_list = tech_stack if tech_stack else ["python"]

Hi! I am Your Hiring Assistant


What is your full name?  bf hwkjf
What is your mobile number?  9300494
What is your email address?  skfjnjsfn
How many years of experience do you have?  2
What position are you applying for?  web developer
Tech Stack: what are your technical skills (languages, frameworks, databases, tools)?  java,flask,mysql


In [22]:
# Generate questions
questions = []
aspects = [
    "core technical concepts (e.g., syntax, architecture, or mechanisms)",
    "practical coding or implementation (e.g., writing functions, components, or queries)",
    "debugging or performance optimization (e.g., fixing issues or improving efficiency)"
]
for i, aspect in enumerate(aspects):
    tech = tech_list[i % len(tech_list)]  # Cycle through techs
    prompt = f"""
    You are a technical hiring assistant for a recruitment agency. Generate **one technical interview question** to evaluate a candidate's proficiency in {tech}. The question must:
    - Focus strictly on {aspect}.
    - Be highly specific to {tech}’s technical features, avoiding general or experience-based queries (e.g., do not ask about duration of use).
    - Suit the experience level (beginner for <2 years, intermediate for 2-5 years, advanced for >5 years).
    - Align with the position: {user_data['jobrole']}.
    - Be concise and testable in an interview.

    Candidate Details:
    - Experience: {user_data['exp']}
    - Position: {user_data['jobrole']}

    Output Format:
    {i+1}. [Question]
    """
    inputs = token(prompt, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=200,      # Increased for detail
            num_beams=10,       # Higher quality
            temperature=0.9,    # Balanced diversity
            top_p=0.9,
            no_repeat_ngram_size=2,
            early_stopping=True
        )
    question = token.decode(outputs[0], skip_special_tokens=True)
    questions.append(question.strip())

D:\Hiring Chatbox\env\Lib\site-packages\transformers\generation\configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
D:\Hiring Chatbox\env\Lib\site-packages\transformers\generation\configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [23]:
# Parse and ensure 3 questions
parsed_questions = []
for q in questions:
    q = q.strip()
    if q and (q.startswith(("1.", "2.", "3.")) or len(q) > 20):  # Accept longer non-numbered questions
        parsed_questions.append(q if q.startswith(("1.", "2.", "3.")) else f"{len(parsed_questions)+1}. {q}")

if len(parsed_questions) != 3:
    # Fallback
    tech_list = tech_list if tech_list else ["python"]
    parsed_questions = [
        f"1. Explain a core concept of {tech_list[0]}.",
        f"2. Write a simple function using {tech_list[min(1, len(tech_list)-1)]}.",
        f"3. How would you debug an issue in {tech_list[min(2, len(tech_list)-1)]}?"
    ][:3]

In [24]:
print("\nTechnical Questions:")
print("\n".join(parsed_questions))


Technical Questions:
1. How long have you been using java?
2. How long have you been using flask?
3. How long have you been using mysql?
